In [2]:
import os
import ember
import lightgbm as lgb
import numpy as np
import pandas as pd
binary_name = "mimikatz.exe" #set this to the binary name, put the ones u want to test in /binaries

data_dir = "../ember2018/"

In [3]:
binary_location = f"/workspaces/torment-nexus/binaries/{binary_name}"

def get_feature_names() -> list[str]:
    byte_histogram = [f"Byte Histogram {a}" for a in range(256)] #256

    byte_entropy_histogram = [f"Byte Entropy Histogram {a}" for a in range(256)] #256

    strings_1 = [f"string.{a}" for a in ["numstrings", "avlength", "printables"]]
    strings_2 = [f"string.printabledist_{b}" for b in range(96)]
    strings_3 = [f"string.{a}" for a in ["entropy", "paths", "urls", "registry", "MZ"]] # 8 + 96
    strings = np.concatenate((strings_1,strings_2,strings_3))

    general_info = [f"general.{a}" for a in ["size", "vsize", "has_debug", "exports", "imports", "has_relocations", "has_resources", "has_signature", "has_tls", "symbols"]]

    header_coff = ["header.coff.timestamp"]
    header_coff_machine = [f"header.coff.machine_{a}" for a in range(10)]
    header_coff_characteristics = [f"header.coff.characteristic_{a}" for a in range(10)]
    header_coff_subsystem = [f"header.optional.subsystem_{a}" for a in range(10)]
    header_coff_dll_characteristics = [f"header.optional.dll_characteristic_{a}" for a in range(10)]
    header_coff_magic = [f"header.optional.magic_{a}" for a in range(10)]
    header_optional = [f"header.optional.{a}" for a in ["major_image_version", "minor_image_version", "major_linker_version", "minor_linker_version", "major_operating_system_version", "minor_operating_system_version", "major_subsystem_version", "minor_subsystem_version", "sizeof_code", "sizeof_headers", "sizeof_heap_commit"]] #12
    header = np.concatenate((header_coff,header_coff_machine,header_coff_characteristics,header_coff_subsystem,header_coff_dll_characteristics,header_coff_magic,header_optional))

    sections_general = [f"sections.{a}" for a in ["section_count", "num_empty_sections", "num_unnamed_sections", "num_read_execute_sections", "num_write_sections",]] #JUST general
    sections_section_sizes = [f"sections.section_{a}_size" for a in range(50)] # this messes with hashing which i will understand at a later time
    sections_section_entropy = [f"sections.section_{a}_entropy" for a in range(50)]
    sections_section_vsize = [f"sections.section_{a}_vsize" for a in range(50)]
    sections_entry_name = [f"sections.entry_name_{a}" for a in range(50)]
    sections_characteristics = [f"sections.characteristics_{a}" for a in range(50)]
    sections = np.concatenate((sections_general, sections_section_sizes, sections_section_entropy, sections_section_vsize, sections_entry_name, sections_characteristics))

    imports_libraries = [f"imports.libraries.library_{a}" for a in range(256)]
    imports_imports = [f"imports.import_{a}" for a in range(1024)]
    imports = np.concatenate((imports_libraries,imports_imports))

    exports = [f"exports.export_{a}" for a in range(128)]

    name_order = [a.lower() for a in ["EXPORT_TABLE", "IMPORT_TABLE", "RESOURCE_TABLE", "EXCEPTION_TABLE", "CERTIFICATE_TABLE","BASE_RELOCATION_TABLE", "DEBUG", "ARCHITECTURE", "GLOBAL_PTR", "TLS_TABLE", "LOAD_CONFIG_TABLE","BOUND_IMPORT", "IAT", "DELAY_IMPORT_DESCRIPTOR", "CLR_RUNTIME_HEADER"]]
    data_directories_unflat = [[f"directories.{a}_size", f"directories.{a}_vaddress"] for a in name_order]
    data_directories = [item for sublist in data_directories_unflat for item in sublist]


    feature_names = np.concatenate((byte_histogram, byte_entropy_histogram, strings, general_info, header, sections, imports, exports, data_directories))
    return feature_names

def classify_binary(binary_location:str) -> float:
    lgbm_model = lgb.Booster(model_file=os.path.join(data_dir, "ember_model_2018.txt"))
    extractor2 = ember.PEFeatureExtractor(2)

    file_data = open(binary_location, "rb").read()
    feature_vector = extractor2.feature_vector(file_data)

    return lgbm_model.predict([np.array(feature_vector, dtype=np.float32)])[0]

def classify_vectors(feature_vector:np.ndarray | pd.DataFrame) -> float:
    if type(feature_vector) is pd.DataFrame:
        feature_vector = feature_vector.to_numpy()

    lgbm_model = lgb.Booster(model_file=os.path.join(data_dir, "ember_model_2018.txt"))
    return lgbm_model.predict([np.array(feature_vector, dtype=np.float32)])[0]

def get_vectors(binary_location:str) -> np.ndarray:
    extractor2 = ember.PEFeatureExtractor(2)
    file_data = open(binary_location, "rb").read()
    return extractor2.feature_vector(file_data)

def get_dataframe(feature_vector:np.ndarray) -> pd.DataFrame:
    df = pd.DataFrame(feature_vector).T
    df.columns = get_feature_names()
    return df

In [4]:
get_dataframe(get_vectors(binary_location))

Unable to find the section associated with EXPORT_TABLE
Can't read the export table at 0xffffffff


,Byte Histogram 0,Byte Histogram 1,Byte Histogram 2,Byte Histogram 3,Byte Histogram 4,Byte Histogram 5,Byte Histogram 6,Byte Histogram 7,Byte Histogram 8,Byte Histogram 9,...,directories.load_config_table_size,directories.load_config_table_vaddress,directories.bound_import_size,directories.bound_import_vaddress,directories.iat_size,directories.iat_vaddress,directories.delay_import_descriptor_size,directories.delay_import_descriptor_vaddress,directories.clr_runtime_header_size,directories.clr_runtime_header_vaddress
0,0.206864,0.008583,0.00417,0.00469,0.007864,0.002909,0.002785,0.00268,0.00986,0.001998,...,0.0,0.0,0.0,0.0,2416.0,610304.0,96.0,912092.0,0.0,0.0


In [35]:

def objective_function(df):
    return classify_vectors(df)

df = get_dataframe(get_vectors(binary_location))

num_particles = 40
num_iterations = 30
dim = df.shape[1]
c1 = c2 = 1.4
w = 0.7
boundsdict = {"header.coff.timestamp": (0, 0xFFFFFFFF),
              "directories.certificate_table_size": (0,0xFFFFFFFF),
              "directories.debug_vaddress":(0,0xFFFFFFFF),
              "directories.certificate_table_vaddress": (0,0xFFFFFFFF),
              "directories.export_table_vaddress":(0,0xFFFFFFFF),
              "directories.export_table_size":(0,0xFFFFFFFF),
              }
changeable_str = ["header.coff.timestamp",
                  "directories.certificate_table_size",
                  "directories.debug_vaddress",
                  "directories.certificate_table_vaddress",
                  "directories.export_table_vaddress",
                  "directories.export_table_size"]
bounds = []
changeable = []
for index,feature in enumerate(df):
    if feature not in changeable_str:
        bounds.append((df[feature].iloc[0], df[feature].iloc[0]))
    else:
        bounds.append(boundsdict[feature])
        changeable.append(index)

particles = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], (num_particles, dim))
velocities = np.random.uniform(-1, 1, (num_particles, dim))
personal_best_positions = np.copy(particles)
personal_best_scores = np.array([objective_function(p) for p in particles])
global_best_position = personal_best_positions[np.argmin(personal_best_scores)]


for iteration in range(num_iterations):
    for i in range(num_particles):
        for j in changeable:
            velocities[i, j] = (w * velocities[i, j] +
                                c1 * np.random.rand() * (personal_best_positions[i, j] - particles[i, j]) +
                                c2 * np.random.rand() * (global_best_position[j] - particles[i, j]))
            particles[i, j] += velocities[i, j]
            particles[i, j] = np.clip(particles[i, j], bounds[j][0], bounds[j][1])
        score = objective_function(particles[i])
        if score < personal_best_scores[i]:
            personal_best_scores[i] = score
            personal_best_positions[i] = particles[i]
            
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    print(f"Score so far: {min(personal_best_scores)}, iteration #{iteration+1}")

print("Best position:", global_best_position)
print("Best score:", objective_function(global_best_position))


Unable to find the section associated with EXPORT_TABLE
Can't read the export table at 0xffffffff


Score so far: 0.001244033473518713, iteration #1
Score so far: 0.001244033473518713, iteration #2
Score so far: 0.0011011673937120121, iteration #3
Score so far: 0.0011011673937120121, iteration #4
Score so far: 0.0011011673937120121, iteration #5
Score so far: 0.0011011673937120121, iteration #6
Score so far: 0.0011011673937120121, iteration #7
Score so far: 0.0011011673937120121, iteration #8
Score so far: 0.0011011673937120121, iteration #9
Score so far: 0.0011011673937120121, iteration #10
Score so far: 0.0011011673937120121, iteration #11
Score so far: 0.0011011673937120121, iteration #12
Score so far: 0.0011011673937120121, iteration #13
Score so far: 0.0011011673937120121, iteration #14
Score so far: 0.0011011673937120121, iteration #15
Score so far: 0.0011011673937120121, iteration #16
Score so far: 0.0011011673937120121, iteration #17
Score so far: 0.0011011673937120121, iteration #18
Score so far: 0.0011011673937120121, iteration #19
Score so far: 0.0011011673937120121, itera

In [36]:
particledf = pd.DataFrame(particles)
particledf.columns = get_feature_names()

In [39]:

particledf.iloc[:,changeable]

,header.coff.timestamp,directories.export_table_size,directories.export_table_vaddress,directories.certificate_table_size,directories.certificate_table_vaddress,directories.debug_vaddress
0,1.073167e+09,1.720680e+09,1.539164e+09,3.711304e+09,3.940004e+09,4.294967e+09
1,1.061008e+09,1.723396e+09,1.796883e+09,3.535246e+09,3.859908e+09,4.294967e+09
2,9.458849e+08,1.744673e+09,1.474570e+09,3.590617e+09,4.168636e+09,4.294967e+09
3,1.039778e+09,1.911093e+09,1.658729e+09,3.563607e+09,4.281010e+09,4.273397e+09
4,1.061863e+09,2.287140e+09,2.125474e+09,3.498583e+09,3.058766e+09,4.294967e+09
5,1.056661e+09,1.675249e+09,1.519487e+09,3.113465e+09,3.514786e+09,4.064874e+09
6,9.885415e+08,1.578192e+09,1.543576e+09,4.294967e+09,4.126843e+09,4.294967e+09
7,1.154015e+09,1.685407e+09,2.167079e+09,3.674551e+09,3.872980e+09,4.294967e+09
8,1.046956e+09,1.744589e+09,1.592247e+09,3.505593e+09,3.887555e+09,4.294967e+09
9,1.067295e+09,3.364126e+08,1.561089e+09,3.502956e+09,3.577181e+09,4.294967e+09


In [22]:
particledf.columns.get_loc("header.coff.timestamp")


626

In [41]:
particledf["header.coff.timestamp"][9]

1067294718.0893644

In [45]:
bestpositiondf = pd.DataFrame(global_best_position).T
bestpositiondf.columns = get_feature_names()

0    1.071294e+09
Name: header.coff.timestamp, dtype: float64

In [52]:
bestpositiondf.iloc[:,changeable]

,header.coff.timestamp,directories.export_table_size,directories.export_table_vaddress,directories.certificate_table_size,directories.certificate_table_vaddress,directories.debug_vaddress
0,1.071294e+09,1.699219e+09,1.535094e+09,3.585796e+09,3.936010e+09,4.294967e+09
